In [7]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [8]:
#load data
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

#set up size of validation, train, and test examples, makes sure they are appropriate dtype 
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

#function to keep image data manageable, expectable
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

#since CPU cant load all images at once, make buffer to handle smaller loads
BUFFER_SIZE = 10000
#shuffle scaled data and segregate
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

#this variable can be used to fine tune model accuracy later
BATCH_SIZE = 100

#we overwrite the train_data here since we won't need unbatched data later.
#we don't need to batch validation data since we don't need to backpropagate weights with validation data
#we do however need to keep track of validation loss
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

#we want the validation data to have same shape and type as train data 
validation_inputs, validation_targets = next(iter(validation_data))

In [9]:
#since our images are broken up bit by bit, and stacked into single column, our input size is 784 (28*28)
#hidden_layer_size is 
input_size = 784
output_size = 10
hidden_layer_size = 50


model = tf.keras.Sequential([
    #since I don't yet know CNNs I use a feed forward nueral network, which takes a single vector
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
    
    
])

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
NUM_EPOCHS = 5

model.fit(train_data,epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 4s - loss: 0.4065 - accuracy: 0.8881 - val_loss: 0.2195 - val_accuracy: 0.9347 - 4s/epoch - 7ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1876 - accuracy: 0.9459 - val_loss: 0.1740 - val_accuracy: 0.9462 - 2s/epoch - 3ms/step
Epoch 3/5
540/540 - 2s - loss: 0.1396 - accuracy: 0.9587 - val_loss: 0.1378 - val_accuracy: 0.9580 - 2s/epoch - 3ms/step
Epoch 4/5
540/540 - 2s - loss: 0.1133 - accuracy: 0.9664 - val_loss: 0.1123 - val_accuracy: 0.9673 - 2s/epoch - 3ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0954 - accuracy: 0.9714 - val_loss: 0.0981 - val_accuracy: 0.9717 - 2s/epoch - 3ms/step
